In [61]:
import PyPDF2
import json
import glob
import numpy as np
import pdfplumber
import pandas as pd

from itertools import chain
from collections import defaultdict
from pdfminer.high_level import extract_pages, extract_text
from pdfminer.layout import LTTextContainer, LTChar, LTRect, LTFigure

In [2]:
def monthToNum(shortMonth):
    return {
            'January': "01",
            'February': "02",
            'March': "03",
            'April': "04",
            'May': "05",
            'June': "06",
            'July': "07",
            'August': "08",
            'September': "09", 
            'October': "10",
            'November': "11",
            'December': "12"
    }[shortMonth]

def defaultify(d):
    
    # https://stackoverflow.com/questions/50013768/how-can-i-convert-nested-dictionary-to-defaultdict
    
    if not isinstance(d, dict):
        return d
    return defaultdict(lambda: None, {k: defaultify(v) for k, v in d.items()})

In [196]:
path_output=\
'/Volumes/4TB/ConferenceCall/Tesla/csvfile/tesla_earning_call_presentation.json'

In [197]:
with open(path_output, 'r') as openfile:
     json_object = json.load(openfile)
# print(json_object)
# print(type(json_object))

In [198]:
def_object=defaultify(json_object)

In [199]:
data_table=\
pd.DataFrame([
              col1, col2, col3, col4, d4
             ]
              for col1, d1 in def_object.items()\
              for col2, d2 in d1.items()\
              for col3, d3 in d2.items()\
              for col4, d4 in d3.items()
            )
data_table=data_table.rename(columns={0: 'time',
                                      1: 'company',
                                      2: 'chara1',
                                      3: 'chara2',
                                      4: 'text'})

data_table_=data_table[data_table['chara1'] == 'presentation_people_speech']

data_table_.loc[:, 'text01']=\
data_table_.loc[:, 'text'].apply(
    lambda x: list(x.values())).apply(
    lambda x: list(chain.from_iterable(x))).apply(
    lambda x: ' '.join(x))

data_table_=data_table_.reset_index(drop=True)

/var/folders/6l/2d59bcb16yjbzzkcgz02zhn00000gn/T/ipykernel_97875/3791956451.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_table_.loc[:, 'text01']=\


In [200]:
_data_table_=data_table_[['time', 'company', 'chara2', 'text01']]

_data_table_=\
pd.pivot(_data_table_,
         values=['text01'],
         index=['time', 'company'],
         columns=['chara2'])

_data_table_=_data_table_['text01'].reset_index(drop=False)

cols = _data_table_.iloc[:,2:].columns
_data_table_['presentationa_all'] = _data_table_[cols].apply(
    lambda x: ' '.join(x.dropna().astype(str)), 1)

data_table_all=_data_table_.loc[:, ['time', 'company', 'presentationa_all']]

In [201]:
data_table_all["call_date"]=data_table_all["time"].apply(lambda x: x.split(',')[1])
data_table_all["call_year"]=\
data_table_all["time"].apply(lambda x: x.split(',')[2][0:5])
data_table_all["call_time"]=\
data_table_all["time"].apply(lambda x: x.split(',')[2][5:11])
data_table_all["quarter"]=\
data_table_all["company"].apply(lambda x: x.split(',')[2][:3])
data_table_all["year_quarter"]=\
data_table_all["company"].apply(lambda x: x.split(',')[2][3:9])
data_table_all["company_name"]=data_table_all["company"].apply(lambda x: x.split(',')[0][0:12])
data_table_all=data_table_all.sort_values(['year_quarter', 'quarter'])
data_table_all=data_table_all.reset_index(drop=True)

In [229]:
data_table_all['call_month']=\
data_table_all['call_date'].apply(
    lambda x: x.split(' ')[1]).apply(
    lambda x: monthToNum(x))

In [234]:
data_table_all['call_day']=\
data_table_all['call_date'].apply(
    lambda x: x.split(' ')[2])

In [256]:
cols = data_table_all.loc[:, ['call_year', 'call_month', 'call_day']].columns
data_table_all['call_datetime']=\
data_table_all[cols].apply(
    lambda x: '-'.join(x.dropna().astype(str)), 1)

In [259]:
data_table_all.columns

Index(['time', 'company', 'presentationa_all', 'call_date', 'call_year',
       'call_time', 'quarter', 'year_quarter', 'company_name', 'call_month',
       'call_day', 'call_datetime'],
      dtype='object', name='chara2')

In [266]:
data_table_all=\
data_table_all[['time',
                'call_datetime',
                'call_year',
                'call_month',
                'call_day',
                'company_name',
                'company',
                'quarter',
                'year_quarter',
                'presentationa_all']]

In [270]:
data_table_all.head()

chara2,time,call_datetime,call_year,call_month,call_day,company_name,company,quarter,year_quarter,presentationa_all
0,"Tuesday, November 09, 2010 10:00 PM GMT",2010-11-09,2010,11,09,Tesla Motors,"Tesla Motors, Inc., Q3 2010 Earnings Call, Nov...",Q3,2010,All right. So as I mentioned in the last earni...
1,"Tuesday, February 15, 2011 10:00 PM GMT",2011-02-15,2011,02,15,Tesla Motors,"Tesla Motors, Inc., Q4 2010 Earnings Call, Feb...",Q4,2010,"Thank you, Tyrone. And thank you, everyone for..."
2,"Wednesday, May 04, 2011 9:00 PM GMT\nS&P",2011-05-04,2011,05,04,Tesla Motors,"Tesla Motors, Inc., Q1 2011 Earnings Call, May...",Q1,2011,"Thank you, Tyrone, and thank you all for joini..."
3,"Wednesday, August 03, 2011 9:30 PM GMT\n",2011-08-03,2011,08,03,Tesla Motors,"Tesla Motors, Inc., Q2 2011 Earnings Call, Aug...",Q2,2011,"Thank you, Tyrone, and good afternoon, everyon..."
4,"Wednesday, November 02, 2011 9:30 PM GM",2011-11-02,2011,11,02,Tesla Motors,"Tesla Motors, Inc., Q3 2011 Earnings Call, Nov...",Q3,2011,"Thank you, Matthew, and good afternoon, everyo..."


In [268]:
path_output=\
'/Volumes/4TB/ConferenceCall/Tesla/csvfile/tesla_earning_call_presentation_all.csv'

In [271]:
data_table_all.to_csv(path_output, index = False)